<a href="https://colab.research.google.com/github/Maggiey01/Rights-Colab-YH/blob/main/20F_country.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract country information (jurisdiction incorporation address) from each 20F

In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


### Findings:
1. 759 out of 1108 (69%) have valid address/file is not empty  
sufficient to go with jurisdication incorporation adress
2. retrive 47 unique countries from address
3. NA	298 filings (27%) not find jurisdication address,
51 (5%) fillings are empty
  
4. took 2m30s to process 

* 
alter before execute  
'local_download_path' path of all 20F files

In [ ]:
pip install pycountry

     |████████████████████████████████| 10.1 MB 20.6 MB/s 
  Created wheel for pycountry: filename=pycountry-22.1.10-py2.py3-none-any.whl size=10595784 sha256=7dcb2d4b2b0a8f3efb31e882253cb6830dc7b52b8b36079cb140a9909a31bec4
  Stored in directory: /root/.cache/pip/wheels/f7/8f/9c/b070d7376caf2beb0685bf72578106b2fd57019ed57d84f126
Successfully built pycountry


In [ ]:
import pandas as pd
import re
import nltk
import os
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
local_download_path = '/drive/MyDrive/Rights Colab YH/20F'

In [ ]:
# create file list of each file in dir
filelist_20F_21 = []
local_download_path='/drive/MyDrive/Rights Colab YH/'
for filename in os.listdir(local_download_path):

    filename = local_download_path+filename
    filelist_20F_21.append(filename)

In [ ]:
#remove'/drive/MyDrive/Rights Colab YH/20F'
filelist_20F_21.remove('/drive/MyDrive/Rights Colab YH/20F')


In [ ]:
#test 5 file
path1 = '/drive/MyDrive/Rights Colab YH/20FIRS-WT_20-F_0001568276-21-001788'
path2 = '/drive/MyDrive/Rights Colab YH/20F_UNK_20-F_0000950103-21-005896'
path3 = '/drive/MyDrive/Rights Colab YH/20FAACG_20-F_0001564590-21-018581'
path4 = '/drive/MyDrive/Rights Colab YH/20FADSE_20-F_0001213900-21-067972'
path5 = '/drive/MyDrive/Rights Colab YH/20FARGX_20-F_0001558370-21-003722'
# path1 - (Jurisdiction of incorporation or organization) - find last line - 
## path2 - Hong Kong  (Jurisdiction of incorporation or organization)

In [ ]:
#match jurisdiction incorporation address
#if no jurisdiction incorporation address fill NA
def find_state(tmp):
    import re
    reg_add_off = r'address of principal executive offices'
    reg_add_incor = r'jurisdiction of incorporation or organization'
    i=0  
    with tmp as myfile:
    #with open(file_in) as myfile:
      try:
        head = [next(myfile) for x in range(60)]
        for line in head:
          i = i+1
          #only test those with jurisdiction address
          if re.findall(reg_add_incor, line.lower()) != []:
            address = head[i-2:i+10]
            return address
      except (StopIteration,IsADirectoryError) as e:
        print(e)
        


In [ ]:
#create file dataframe
def file_df(filepath):
  tmp_pd = pd.DataFrame()
  try:
    for file in filepath:
      tmp = open(file)
      adress = find_state(tmp)
      tmp_pd = tmp_pd.append({'path':file, 'state': adress}, ignore_index=True)
      tmp_pd['jurisdiction_add'] = tmp_pd['state'].apply(find_country)
    return tmp_pd
  except IsADirectoryError as e:
    print(e)

In [ ]:
#find countries name in state
def find_country(txtadd):
  import pycountry
  if txtadd != None:
    
    for line in txtadd:
      for country in pycountry.countries:
          if country.name in line:
            return country.name
  else:
    return 'NA'         

In [ ]:
#run on all files
file_country = file_df(filelist_20F_21)

In [ ]:
file_country.head(5)

,path,state,jurisdiction_add
0,/drive/MyDrive/Rights Colab YH/20FARCXF_20-F_0...,"[Grand Duchy of Luxembourg\n, (Jurisdiction of...",Luxembourg
1,/drive/MyDrive/Rights Colab YH/20FMTCN_20-F_00...,"[Grand Duchy of Luxembourg\n, (Jurisdiction of...",Luxembourg
2,/drive/MyDrive/Rights Colab YH/20FTCOM_20-F_00...,"[Cayman Islands \n, (Jurisdiction of incorpora...",Cayman Islands
3,/drive/MyDrive/Rights Colab YH/20FERLFF_20-F_0...,"[Province of British Columbia, Canada \n, (Jur...",Canada
4,/drive/MyDrive/Rights Colab YH/20FXTGRF_20-F_0...,"[British Virgin Islands\n, (Jurisdiction of in...",Bahamas


## description of results

In [ ]:
file_country.describe()

,path,state,jurisdiction_add
count,1108,810,1057
unique,1108,628,47
top,/drive/MyDrive/Rights Colab YH/20FARCXF_20-F_0...,[ (Exact Name of Registrants as Specified in T...,NA
freq,1,29,298


In [ ]:
file_country['jurisdiction_add'].unique()

array(['Luxembourg', 'Cayman Islands', 'Canada', 'Bahamas', 'Brazil',
       'NA', 'Russian Federation', 'United Kingdom', 'Netherlands',
       'Greece', 'China', 'Marshall Islands', 'Germany', 'Israel', None,
       'Bermuda', 'Denmark', 'France', 'Switzerland', 'Belgium', 'Mexico',
       'Guernsey', 'Singapore', 'Cyprus', 'Georgia', 'Sweden', 'Jersey',
       'Spain', 'Australia', 'Indonesia', 'Italy', 'Chile', 'India',
       'Japan', 'Colombia', 'Hong Kong', 'Argentina', 'Ireland', 'Peru',
       'Anguilla', 'Panama', 'Uruguay', 'Gibraltar', 'United States',
       'Norway', 'Philippines', 'Mauritius', 'South Africa'], dtype=object)

In [ ]:
country_des = file_country.groupby('jurisdiction_add').count()

In [ ]:
country_des

,path,state
jurisdiction_add,,
Anguilla,1,1
Argentina,14,14
Australia,11,11
Bahamas,2,2
Belgium,4,4
Bermuda,23,23
Brazil,20,20
Canada,64,64
Cayman Islands,186,186


In [ ]:
country_des['max_rank'] = country_des['state'].rank(method='max')
country_des.sort_values(by=['state'],ascending = False)
excludelist = ['Cayman Islands','Marshall Islands', 'Bermuda','Luxembourg','Ireland','Jersey','Panama','Bahamas','Mauritius',
               'Guernsey','Cyprus','Gibraltar','Anguilla','NA']

In [ ]:
country_focus = country_des[~country_des['state'].isin(excludelist)]
#df[df['country'] not in countries_to_keep]
#df[~df['countries'].isin(c1)]

In [ ]:
country_focus

,path,state,max_rank
jurisdiction_add,,,
Anguilla,1,1,9.0
Argentina,14,14,34.0
Australia,11,11,32.0
Bahamas,2,2,14.0
Belgium,4,4,22.0
Bermuda,23,23,39.0
Brazil,20,20,38.0
Canada,64,64,45.0
Cayman Islands,186,186,47.0


find pattern of country




1. a line before (Jurisdiction of Incorporation or Organization|Jurisdiction of incorporation)
2. same line with (Jurisdiction of Incorporation or Organization|Jurisdiction of incorporation)
3. no (Jurisdiction of Incorporation or Organization|Jurisdiction of incorporation) but (Address of principal executive offices)
4. it took 3min to go through all text - try to limit to head50 line